# Mast cells

### read in Mast reference

In [ ]:
Mast_scffpe = readr::read_rds('cleaned_reference_Mast_cells.rds')
Mast_scffpe

In [ ]:
Mast_scffpe@meta.data$technology = 'scffpe'

In [ ]:
Mast_scffpe@meta.data$coarse_ids = Mast_scffpe@meta.data$cleaned_coarse_ids 

## subset xenium to Mast cells

In [ ]:
Mast_xenium = subset(L79_E, coarse_ids == 'Mast')
Mast_xenium

## and to the selected genes

In [ ]:
Mast_xenium = Mast_xenium[rownames(Mast_scffpe),]
Mast_xenium

In [ ]:
Mast_xenium@meta.data$technology = 'xenium'

## merge datasets

In [ ]:
Mast_merged = merge(Mast_scffpe, Mast_xenium)
Mast_merged

## normalize, scale, pca,

In [ ]:
normFactor = Mast_merged@meta.data %>%
    select(technology, nCount_RNA, orig.ident) %>%
    group_by(technology, orig.ident) %>%
    summarize(medianCounts = median(nCount_RNA)) 
    normFactor = mean(normFactor$medianCounts) #median of medians (across technology and orig.ident) 

In [ ]:
Mast_merged = NormalizeData(Mast_merged, 
                          normalization.method = "LogNormalize", 
                          scale.factor = normFactor) %>%
    ScaleData(features = rownames(Mast_merged)) %>%
    singlecellmethods::RunBalancedPCA(weight.by='orig.ident', npcs=15)
Mast_merged

## harmonize

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
Mast_merged = Mast_merged %>% harmony::RunHarmony(c("orig.ident", "technology"), 
    plot_convergence = FALSE, 
    lambda = NULL,
    max.iter = 10,
    early_stop = F
) 

## transfer labels

In [ ]:
Mast_merged = TransferLabels(Mast_merged, 
                               reduction = 'harmony', 
                               group.by.varname = 'technology', 
                               label.varname = 'fine_ids', 
                               from = 'scffpe', 
                               to = 'xenium', 
                               k = 20, 
                               weighted = TRUE)
Mast_merged

## umap

In [ ]:
U = uwot::umap( Mast_merged@reductions$harmony@cell.embeddings[, 1:15], 
               min_dist = 0.05, 
               spread = 0.30, 
               ret_extra = 'fgraph', 
               n_sgd_threads = nbrOfWorkers(), 
               fast_sgd = TRUE)
colnames(U$embedding) = c('HUMAP1', 'HUMAP2')
rownames(U$fgraph) = colnames(U$fgraph) = Cells( Mast_merged)
 Mast_merged[['humap']] = Seurat::CreateDimReducObject(
    embeddings = U$embedding,
    assay = 'RNA', 
    key = 'HUMAP_', 
    global = TRUE
)
new_graph = Seurat::as.Graph(U$fgraph)
DefaultAssay(new_graph) = DefaultAssay( Mast_merged)
 Mast_merged[['humap_fgraph']] = new_graph

In [ ]:
Mast_merged@meta.data %>% colnames

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 15)
Mast_merged %>% Embeddings(reduction = 'humap') %>% as.data.frame %>% mutate(cell = rownames(.)) %>% left_join(Mast_merged@meta.data %>% mutate(cell = rownames(.))) %>% select(cell, HUMAP_1, HUMAP_2, coarse_ids, fine_ids, technology) %>% ggplot() +
geom_point(aes(HUMAP_1, HUMAP_2, color = fine_ids), shape = '.') +
facet_wrap(~fine_ids + technology) +
gghighlight::gghighlight()

## cache

In [ ]:
readr::write_rds(x = Mast_merged, file = 'Mast_L79_E.rds')

In [ ]:
rm(Mast_merged)
rm(Mast_scffpe)
rm(Mast_xenium)

In [ ]:
L79_E = readr::read_rds('/n/data1/bwh/medicine/korsunsky/lab/data/10Xgenomics/Xenium_20240219//1st_round/20231202__213743__BWH_20231202_lung/output-XETG00150__0014607__L79-E__20231202__213943/baysor_without_controls/merged.rds')

In [ ]:
unique(L79_E@meta.data$cleaned_coarse_ids)

In [ ]:
'Epithelial' = done
'T' = done
'Fibroblast' = done
'Myeloid' = done
'Plasma' = left out
'Endothelial' = done
'B' = left out
'Mast' = done
'Mural' = done